In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install nptdms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nptdms: filename=nptdms-1.10.0-py3-none-any.whl size=108456 sha256=9f0de9b3ec5b850d1a2c9f746db92d4fde91bf51f50d122f544a510e1c07491c
  Stored in directory: /root/.cache/pip/wheels/1b/4b/17/21e8b03b37ea51ce7ec9f5570cdf0decca93f537d61c06880f
Successfully built nptdms


## 중단 조건 만족 데이터셋 $T_{failure}$ 처리

파일 불러오기

In [3]:
import os
import pandas as pd
from nptdms import TdmsFile

In [4]:
def load_tdms_file(file_path):
    tdms_file = TdmsFile.read(file_path)

    group_name_vibration = tdms_file.groups()[0].name
    group_name_operation = tdms_file.groups()[1].name

    vib_channels = tdms_file[group_name_vibration].channels()
    vib_data = {ch.name.strip(): ch.data for ch in vib_channels}

    operation_channels = tdms_file[group_name_operation].channels()
    operation_data = {ch.name.strip(): ch.data for ch in operation_channels}

    return vib_data, operation_data

In [5]:
def label_rul_by_stopping_condition(df, interval_sec=600):
    df = df.copy()
    # 중단 조건 인덱스 찾기
    torque_fail_indices = df.index[df['Torque[Nm]'] <= -17]
    temp_fail_indices = df.index[(df['TC SP Front[℃]'] >= 200) | (df['TC SP Rear[℃]'] >= 200)]

    if len(torque_fail_indices) == 0 and len(temp_fail_indices) == 0:
        print("No stopping condition reached.")
        df['RUL_sec'] = None
        return df

    torque_fail_index = torque_fail_indices.min() if len(torque_fail_indices) > 0 else None
    temp_fail_index = temp_fail_indices.min() if len(temp_fail_indices) > 0 else None

    if torque_fail_index is not None and temp_fail_index is not None:
        T_failure_index = min(torque_fail_index, temp_fail_index)
    elif torque_fail_index is not None:
        T_failure_index = torque_fail_index
    elif temp_fail_index is not None:
        T_failure_index = temp_fail_index
    else:
        T_failure_index = None

    if T_failure_index is None:
        print("No failure index found.")
        df['RUL_sec'] = None
        return df

    df['RUL_sec'] = (T_failure_index - df.index) * interval_sec
    df['RUL_sec'] = df['RUL_sec'].clip(lower=0)

    print(f"Failure index at {T_failure_index}, time {T_failure_index * interval_sec} sec")
    return df

In [6]:
# 데이터셋 경로와 타겟 폴더 설정
train_root = "/content/drive/MyDrive/KSPHM-data-challenge/Train Set"
target_folders = ["Train1", "Train4", "Train5", "Train7", "Train8"]

summary_stopping = {}

for folder in target_folders:
    folder_path = os.path.join(train_root, folder)
    tdms_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".tdms")])

    records = []

    for idx, tdms_file in enumerate(tdms_files):
        file_path = os.path.join(folder_path, tdms_file)
        _, operation_data = load_tdms_file(file_path)

        # 각 파일 첫 번째 측정값만 사용 (index는 10분 단위)
        time_sec = idx * 600

        record = {
            "time_sec": time_sec,
            "Torque[Nm]": operation_data["Torque[Nm]"][0],
            "TC SP Front[℃]": operation_data["TC SP Front[℃]"][0],
            "TC SP Rear[℃]": operation_data["TC SP Rear[℃]"][0],
        }
        records.append(record)

    df = pd.DataFrame(records)

    # RUL 라벨링 적용
    df_labeled = label_rul_by_stopping_condition(df, interval_sec=600)
    summary_stopping[folder] = df_labeled

    # 결과 출력
    print(f"=== {folder} 라벨링 결과 ===")
    print(df_labeled.tail(5))
    print()

Failure index at 93, time 55800 sec
=== Train1 라벨링 결과 ===
    time_sec  Torque[Nm]  TC SP Front[℃]  TC SP Rear[℃]  RUL_sec
89     53400   -7.372215        100.6211       153.8347     2400
90     54000   -6.857190        101.3231       156.6417     1800
91     54600   -8.219799        100.5940       181.6120     1200
92     55200   -7.352595        102.5969       175.5891      600
93     55800  -17.921889        106.0521       199.1974        0

Failure index at 221, time 132600 sec
=== Train4 라벨링 결과 ===
     time_sec  Torque[Nm]  TC SP Front[℃]  TC SP Rear[℃]  RUL_sec
217    130200   -7.744014        132.3273       114.2068     2400
218    130800   -8.043219        134.1012       113.9421     1800
219    131400   -8.661249        138.0506       113.8825     1200
220    132000   -9.538263        146.1215       113.6777      600
221    132600  -17.102754        198.7421       114.3982        0

Failure index at 260, time 156000 sec
=== Train5 라벨링 결과 ===
     time_sec  Torque[Nm]  TC SP F